![](logo.jpeg)

# 1. Introduction

Continuous advancements in Unmanned Aerial Vehicle (UAV) technology have broadened its applications in agriculture. These innovations play a crucial role in fostering sustainable solutions and influencing policies and decisions in agriculture. However, the regulatory framework for UAV flights and the development of standardized protocols for precise data capture are continuously evolving. It is imperative to uphold accurate data quality for synergizing multiyear data and other spectral sensors (satellites, drones, lab sensors. To maintain high standards, some essential steps were proposed. 

# 2. Pre planning
## 2.1. Flight planning
* To carry successful drone surveys, a strategic flight planning is required using a standard flight planning software. Coordinate with the project representative to establish suitable flight parameters, including spatial resolution, date, and additional ancillary measurements. 

* Ensure the survey area is not within classfied airspace.

* List the equipment required for the survey. 

* Select clearly identifiable ground control point (GCP) locations within the survey area, essential for referencing UAV products.
## 2.2. Weather forecasting
Verify the weather forecast before initiating the survey.
Increasing availability atmosphere data from multiple satellites enables to predict accurate weather prediction for drone campaigns. 

Accessing weather information from ECMWF forecasts and Meteosat imagery allows the user to view up to two synchronised loops of the products. 

If possible, obtain local/in-situ weather conditions (cloud cover, wind etc.) from the site person. 
## 2.3. Site and risk assessment and permissions
The pilot should identify the site using given coordinates in Google earth Maps or ArcGIS. 

Identify geographical features (big trees, bird conservation parks, tall building, electric polls/wires) around the survey area which might affect the data (electromagnetic interference) and flight planning.   

Identify possible hazards within the survey area. 

The restrictions (permanent and temporary) over the site should be checked. 

To ensure lawful and ethical operations, the site permission should be obtained from the project representative/ site manager.  
## 2.4. Flight survey time 

* Record the site conditions in the logbook before initiating the drone flight. 

* Check the GPS/GNSS quality, ensuring a minimum of 5 satellites are available. 

* Place the GCP targets in pre-defined locations. 

* Ensure sensors lens is clean if not clean with a lens tissue. 

* Turn ON instruments, especially the spectral camera, 15 minutes before collecting actual measurements. 

* Ensure that a clean calibration target should be used for the calibration. 

* It is advisable to record the calibration target before after the survey. If the survey area is big, additional measurements are required. 

* Record essential parameters in metadata sheet (Table 1) and logbook. 

## 2.5. Ancillary measurements to support UAV acquisitions. 
Calculating accurate surface reflectance values are essential in various applications and to synergies with other data. To support this, additional measurements can be conducted. 

**Field spectroradiometer**

* Simultaneously with the drone flight, ground spectral measurements can be acquired using a spectroradiometer to align with drone spectral data. 

* Downwelling solar radiation can be captured with a spectroradiometer, utilizing a sensor fiber attached to a cosine receptor and positioned upward with a tripod. 

**Sun photometer** for measuring aerosol optical thickness.

# 3. Data management
After the drone flight, cross checking is performed to ensure detailed metadata is recorded, and data is stored in an appropriately. Effective data management is crucial for subsequent analysis. 

Visual evaluation of images is necessary to identify undesirable aspects such as blurry images (RGB), illumination variations, and naming issues. 

# 4. Data processing piepleines

The radiometric response of optical sensors/ remote sensing imagery play critical role in obtaining quantitative spectral infromation. Typically, the data from the spectral sensors can be delivered as digital numbers, radiance, and reflectance. 

* Digital numbers(DN) The amount of reflected light energy measured by the sensor recorded in the form of a binary integers which is not considered as qunatitative value thus not scientifc value. The range of binary integers depend on the radiometric resolution of the sensor. For example A sensor with 8 bits (rediometric resolution) records the data from 0 to 255, while 16 bits sensor records values from 0 to 65536.
Radiometric resolution indciates the sensor sensitivity to detect minor energy differences. High resolution sensors has higher sensitivity.



# 4.1. Converting Digital numbers to Radiance

Radiance The DN converted to radiance using the a gain and offset values per each band, this process also called as radiometric calibration. The gain and offset values generally supplied from teh sensor manufactrer, if not the values can be retrieved using an integrating sphere. The radaince units are watts/m2 steradian nm.

$$
L_{band} = DN * Gain_{band} + Offset_{band}
$$

where: 

* L_band = spectral radaince c(watts/(m2*steradian nm)
* DN = Digital numbers
* Gain =  Slope of the spectral band
* Offset = Intercept of the spectral band


# 4.2. Radiance to Reflectance

The portional of the light reflected from the object to the sensors. The reflectance measured in the range of 0-100% which depend on the properties of the reflected surface. Since the radiance values infleunced by various components (atmospheric and aerosol profile, and geometric factors) their interactions with radition, conversion into surface relfectance is essential to retrive phenological infromation.
Generally three methods are followed these effects.

* Empirical Line
* Atmospheric Radiative transfer model (RTM)
* Hybrid method

Empirical appraoches are widely used because of ease in implementation and repeatable under constant illumination conditions based on linear regression between reference panels and drone radiance values.
Reading the raw multispectral images from the drone and and calibrating them into reflectance maps and stitiching the phtos can be done using different photorammetry softwares (Pix4D, Agrisoft, ArcGIS Drone2map).
In case of multispectral images stadard reference panels (white and dark) are used to determine the scale factor which later applied over the image to convert into reflectance cube. The above softwares automatically extract reference panel information from by finding the QR code within an image. 

# 4.3. Deriving vegetation indices

  

In [28]:
import numpy as np
import rasterio

image_array = rasterio.open('data/Multispec_30m.tif')
image_array.meta

{'driver': 'GTiff',
 'dtype': 'float32',
 'nodata': -32767.0,
 'width': 3529,
 'height': 3432,
 'count': 6,
 'crs': CRS.from_epsg(7853),
 'transform': Affine(0.020399847655346234, 0.0, 433027.87620212696,
        0.0, -0.020401199596840672, 6378518.849946602)}

In [30]:
# Spectral Bands

green = image_array.read(1)
blue = image_array.read(2)
red = image_array.read(3)
rededge = image_array.read(3)
nir = image_array.read(3)
panchromatic = image_array.read(3)

#NDVI - Normalized Vegetation Index
NDVI = (nir.astype(float) - red.astype(float)) / (nir + red)

#RVI - Simple Ratio Vegetation Index
RVI = nir.astype(float)/red.astype(float)

#SAVI - Soil Adjusted Vegetation Index
L= 0.5
SAVI = (nir.astype(float) - red.astype(float))/(nir.astype(float) + red.astype(float) + L)

# MSAVI2 - Modified Secondary Soil Adjusted Vegetation Index

MSAVI2 = 0.5 * (2 * nir.astype(float) + 1 - np.sqrt(2 * nir.astype(float) + 1) * 2 - 8*(nir.astype(float) - red.astype(float)))
                                                               
# ARVI - Atmospheric Resistant Vegetation Index
ARVI = (nir.astype(float) + blue.astype(float))/(nir.astype(float) - blue.astype(float))


C:\Users\a1233249\AppData\Local\Temp\ipykernel_31296\866925832.py:22: RuntimeWarning: invalid value encountered in sqrt
  MSAVI2 = 0.5 * (2 * nir.astype(float) + 1 - np.sqrt(2 * nir.astype(float) + 1) * 2 - 8*(nir.astype(float) - red.astype(float)))
C:\Users\a1233249\AppData\Local\Temp\ipykernel_31296\866925832.py:25: RuntimeWarning: divide by zero encountered in divide
  ARVI = (nir.astype(float) + blue.astype(float))/(nir.astype(float) - blue.astype(float))


There is a lot more that you can do with outputs (such as including interactive outputs)
with your book. For more information about this, see [the Jupyter Book documentation](https://jupyterbook.org)